# 🚀 langchain_genai — Comprehensive Usage Examples

This notebook demonstrates every capability of the `langchain_genai` package:

| # | Section | Capability |
|---|---------|-----------|
| 1 | **Setup** | Imports & model initialization |
| 2 | **Basic Chat** | `invoke` with messages, usage metadata |
| 4 | **Custom Tools** | `@tool` decorator, `bind_tools`, tool calling |
| 5 | **Structured Output** | `with_structured_output`, Pydantic schemas, `include_raw` |
| 6 | **Embeddings** | `embed_documents`, `embed_query`, cosine similarity |
| 7 | **Async** | `ainvoke`, `aembed_documents` |
| 8 | **ReAct Agent** | `create_react_agent` with tool loop |
| 9 | **Multi-turn Agent** | Agent with conversation memory |
| 10 | **Advanced Agent** | Multi-step tool chaining, sequential execution |

## 1. Setup & Imports

In [1]:
# Core imports — langchain_genai package
from langchain_genai import GenAIChatModel, GenAIEmbeddings

# LangChain message types
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

# Chains & prompts
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

# Tools
from langchain_core.tools import tool

# Structured output
from pydantic import BaseModel, Field

print("✅ All imports successful")

✅ All imports successful


In [2]:
# Initialize the chat model (uses config.yaml at repo root by default)
model = GenAIChatModel(model="gpt-4.1-nano")

# Verify it's working
print(f"Model type: {model._llm_type}")
print(f"Model name: {model.model}")

Model type: genai-chat
Model name: gpt-4.1-nano


## 2. Basic Chat — `invoke` with Messages

The simplest usage: send a message and get a response back as an `AIMessage`.

In [3]:
# Simple single-message invocation
response = model.invoke([HumanMessage(content="What is the capital of Japan?")])

print(f"Response: {response.content}")
print(f"Type:     {type(response).__name__}")

Response: The capital of Japan is Tokyo.
Type:     AIMessage


In [4]:
# Multi-turn conversation with system + human messages
response = model.invoke([
    SystemMessage(content="You are a helpful geography expert. Answer concisely."),
    HumanMessage(content="Name the three largest countries by area."),
])

print(response.content)
print(f"\nUsage: {response.usage_metadata}")
print(f"Model: {response.response_metadata.get('model', 'N/A')}")

Russia, Canada, and the United States.

Usage: {'input_tokens': 31, 'output_tokens': 10, 'total_tokens': 41}
Model: N/A


## 4. Tool Calling — `bind_tools`

Define Python functions as **LangChain tools**, bind them to the model, and let the LLM decide which tool to call.

In [6]:
# Define tools
@tool
def get_weather(city: str) -> str:
    """Return the current weather for a city."""
    # Fake implementation for demo purposes
    weather_data = {
        "tokyo": "☀️ 22°C, clear skies",
        "london": "🌧️ 14°C, light rain",
        "new york": "⛅ 18°C, partly cloudy",
    }
    return weather_data.get(city.lower(), f"No data for {city}")

@tool
def calculate(expression: str) -> str:
    """Evaluate a simple math expression and return the result."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

tools = [get_weather, calculate]

# Bind tools to the model
model_with_tools = model.bind_tools(tools)

# Ask a question that should trigger tool usage
response = model_with_tools.invoke([
    HumanMessage(content="What is the weather in Tokyo?")
])

print("Content:", response.content or "(empty — tool call issued)")
print("Tool calls:")
for tc in response.tool_calls:
    print(f"  → {tc['name']}({tc['args']})")

Content: (empty — tool call issued)
Tool calls:
  → get_weather({'city': 'Tokyo'})


In [7]:
# Execute the tool call manually (outside an agent loop)
if response.tool_calls:
    tc = response.tool_calls[0]
    # Look up the tool function by name and call it
    tool_map = {t.name: t for t in tools}
    tool_result = tool_map[tc["name"]].invoke(tc["args"])
    print(f"Tool '{tc['name']}' returned: {tool_result}")

    # Feed the result back to the model using ToolMessage
    followup = model_with_tools.invoke([
        HumanMessage(content="What is the weather in Tokyo?"),
        response,  # The AIMessage with tool_calls
        ToolMessage(content=tool_result, tool_call_id=tc["id"]),
    ])
    print(f"\nFinal answer: {followup.content}")

Tool 'get_weather' returned: ☀️ 22°C, clear skies

Final answer: The weather in Tokyo is currently clear with a temperature of 22°C.


## 5. Structured Output — `with_structured_output`

Extract data from free-text into a strongly-typed **Pydantic model**.

In [8]:
# Define the output schema
class MovieReview(BaseModel):
    """Structured movie review."""
    title: str = Field(description="Movie title")
    rating: float = Field(description="Rating out of 10")
    pros: list[str] = Field(description="List of positive aspects")
    cons: list[str] = Field(description="List of negative aspects")
    recommendation: bool = Field(description="Whether to recommend the movie")

# Create a structured output model
structured_model = model.with_structured_output(MovieReview)

review = structured_model.invoke(
    "Review the movie 'Inception' by Christopher Nolan in detail."
)

print(f"Title:          {review.title}")
print(f"Rating:         {review.rating}/10")
print(f"Pros:           {review.pros}")
print(f"Cons:           {review.cons}")
print(f"Recommendation: {'👍 Yes' if review.recommendation else '👎 No'}")
print(f"\nType: {type(review).__name__}")

Title:          Inception
Rating:         9.0/10
Pros:           ['Intriguing plot with complex layers', 'Excellent direction by Christopher Nolan', 'Strong performances, especially by Leonardo DiCaprio', 'Stunning visual effects and cinematography', 'Thought-provoking themes about reality and subconscious']
Cons:           ['Can be confusing for some viewers due to its complexity', 'Pacing issues in certain parts', 'Requires multiple viewings to fully grasp']
Recommendation: 👍 Yes

Type: MovieReview


In [9]:
# include_raw=True returns both the parsed object and the raw AIMessage
structured_raw = model.with_structured_output(MovieReview, include_raw=True)

raw_result = structured_raw.invoke(
    "Review the movie 'The Matrix' briefly."
)

print("Parsed:", raw_result["parsed"].title, "-", raw_result["parsed"].rating)
print(f"\nRaw AIMessage content (first 200 chars):\n{raw_result['raw'].content[:200]}")

Parsed: The Matrix - 8.7

Raw AIMessage content (first 200 chars):



## 6. Embeddings — `GenAIEmbeddings`

Generate vector embeddings for documents and queries, then compute cosine similarity.

In [11]:
import numpy as np

embeddings = GenAIEmbeddings(model="text-embedding-3-small")

# Embed multiple documents
docs = [
    "LangChain is a framework for building LLM applications.",
    "Python is a popular programming language.",
    "The weather in Tokyo is sunny today.",
]
doc_vectors = embeddings.embed_documents(docs)

print(f"Number of documents: {len(doc_vectors)}")
print(f"Embedding dimension: {len(doc_vectors[0])}")
print(f"First 5 values:      {doc_vectors[0][:5]}")

Number of documents: 3
Embedding dimension: 1536
First 5 values:      [-0.014973461627960205, 0.009841741994023323, 0.04774899408221245, -0.0019219618989154696, 0.0539771243929863]


In [12]:
# Embed a query and find the most similar document via cosine similarity
query = "How do I build apps with large language models?"
query_vector = embeddings.embed_query(query)

def cosine_similarity(a, b):
    a, b = np.array(a), np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

print(f"Query: '{query}'\n")
for i, doc in enumerate(docs):
    sim = cosine_similarity(query_vector, doc_vectors[i])
    print(f"  [{sim:.4f}] {doc}")

best_idx = max(range(len(docs)), key=lambda i: cosine_similarity(query_vector, doc_vectors[i]))
print(f"\n✅ Most similar: '{docs[best_idx]}'")

Query: 'How do I build apps with large language models?'

  [0.3313] LangChain is a framework for building LLM applications.
  [0.2182] Python is a popular programming language.
  [-0.0213] The weather in Tokyo is sunny today.

✅ Most similar: 'LangChain is a framework for building LLM applications.'


## 7. Async Support — `ainvoke` & `aembed`

Both `GenAIChatModel` and `GenAIEmbeddings` support async methods for use in async contexts (web servers, notebooks, etc.).

In [13]:
import asyncio

# Async chat invocation
async_response = await model.ainvoke([
    HumanMessage(content="Explain async programming in one sentence.")
])
print("Async chat:", async_response.content)

# Async embeddings
async_embedding = await embeddings.aembed_query("async programming")
print(f"\nAsync embedding dimension: {len(async_embedding)}")
print(f"First 5 values: {async_embedding[:5]}")

Async chat: Async programming is a programming paradigm that allows tasks to run concurrently by executing code non-blockingly, enabling efficient handling of I/O-bound operations.

Async embedding dimension: 1536
First 5 values: [-0.009581275284290314, -0.015984980389475822, -0.00890135858207941, -0.030249357223510742, 0.04781618341803551]


## 8. ReAct Agent — Tool-Calling Agent with LangGraph

A **ReAct agent** uses a reasoning + acting loop: the LLM decides which tool to call, observes the result, and repeats until it has a final answer.

We use `create_react_agent` from **LangGraph** which handles the full tool-execution loop automatically.

In [14]:
from langgraph.prebuilt import create_react_agent

# Create a ReAct agent with our GenAI model and tools
agent = create_react_agent(model, tools)

# The agent will automatically call get_weather, observe the result, and respond
result = agent.invoke({"messages": [HumanMessage(content="What's the weather like in London?")]})

# Print the full message trace
for msg in result["messages"]:
    role = type(msg).__name__
    content = msg.content or "(tool call)"
    print(f"[{role}] {content[:200]}")
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"  🔧 {tc['name']}({tc['args']})")

C:\Users\likin\AppData\Local\Temp\ipykernel_15828\3219355086.py:4: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model, tools)


[HumanMessage] What's the weather like in London?
[AIMessage] (tool call)
  🔧 get_weather({'city': 'London'})
[ToolMessage] 🌧️ 14°C, light rain
[AIMessage] The weather in London is currently 14°C with light rain.


In [15]:
# Agent with a math question — triggers the calculate tool
result = agent.invoke({
    "messages": [HumanMessage(content="What is 42 * 17 + 89?")]
})

# Show the final answer
final_msg = result["messages"][-1]
print(f"Final answer: {final_msg.content}")

Final answer: The result of 42 multiplied by 17 plus 89 is 803.


## 9. Conversational Agent — Multi-Turn with Memory

Use LangGraph's built-in **thread-based memory** to maintain conversation context across multiple turns. The agent remembers previous exchanges and can reference earlier tool results.

In [16]:
from langgraph.checkpoint.memory import MemorySaver

# Create agent with memory
memory = MemorySaver()
conversational_agent = create_react_agent(model, tools, checkpointer=memory)

# Configuration with a thread ID for conversation tracking
config = {"configurable": {"thread_id": "demo-thread-1"}}

# Turn 1: Ask about weather
print("=" * 60)
print("TURN 1")
print("=" * 60)
r1 = conversational_agent.invoke(
    {"messages": [HumanMessage(content="What's the weather in New York?")]},
    config=config,
)
print(r1["messages"][-1].content)

# Turn 2: Follow-up referencing previous context
print("\n" + "=" * 60)
print("TURN 2 (follow-up)")
print("=" * 60)
r2 = conversational_agent.invoke(
    {"messages": [HumanMessage(content="How about Tokyo? Is it warmer?")]},
    config=config,
)
print(r2["messages"][-1].content)

# Turn 3: Ask something completely different — agent still has context
print("\n" + "=" * 60)
print("TURN 3 (math + context)")
print("=" * 60)
r3 = conversational_agent.invoke(
    {"messages": [HumanMessage(content="What is the temperature difference between those two cities? Calculate it.")]},
    config=config,
)
print(r3["messages"][-1].content)

C:\Users\likin\AppData\Local\Temp\ipykernel_15828\2065884960.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  conversational_agent = create_react_agent(model, tools, checkpointer=memory)


TURN 1
The weather in New York is partly cloudy with a temperature of 18°C.

TURN 2 (follow-up)
The weather in Tokyo is clear with a temperature of 22°C. Yes, it is warmer in Tokyo compared to New York.

TURN 3 (math + context)
The temperature difference between Tokyo and New York is 4°C. Tokyo is 4°C warmer than New York.


## 10. Advanced Agent — Multi-Step Tool Chaining & System Prompt

Create an agent with a **system prompt** that chains multiple tool calls in a single conversation to solve a complex problem.

In [17]:
# Additional tools for a richer agent
@tool
def search_knowledge_base(query: str) -> str:
    """Search the internal knowledge base for information."""
    kb = {
        "langchain": "LangChain is a framework for developing applications powered by LLMs. It supports chains, agents, and retrieval.",
        "langgraph": "LangGraph is a library for building stateful, multi-actor applications with LLMs, built on top of LangChain.",
        "genai": "GenAI refers to generative artificial intelligence systems that can create text, images, and other content.",
        "rag": "RAG (Retrieval-Augmented Generation) combines retrieval from a knowledge base with LLM generation for more accurate answers.",
    }
    for key, value in kb.items():
        if key in query.lower():
            return value
    return f"No results found for '{query}'"

@tool
def format_report(title: str, sections: list[str]) -> str:
    """Format a structured report with a title and sections."""
    report = f"📄 {title}\n{'=' * (len(title) + 3)}\n"
    for i, section in enumerate(sections, 1):
        report += f"\n{i}. {section}"
    return report

# Create an advanced agent with system prompt and expanded tools
advanced_tools = [get_weather, calculate, search_knowledge_base, format_report]

advanced_agent = create_react_agent(
    model,
    advanced_tools,
    prompt="You are a helpful research assistant. Use your tools to gather information, perform calculations, and format results into clear reports. Always be thorough.",
)

# Complex query requiring multiple tool calls
result = advanced_agent.invoke({
    "messages": [HumanMessage(
        content="I need a brief report on what LangChain and LangGraph are. "
                "Search the knowledge base for each, then format the findings into a report titled 'AI Framework Overview'."
    )]
})

# Print full message trace showing multi-step reasoning
print("📋 Agent Message Trace:")
print("-" * 60)
for msg in result["messages"]:
    role = type(msg).__name__
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"[{role}] 🔧 Calling: {tc['name']}({tc['args']})")
    elif role == "ToolMessage":
        print(f"[{role}] ← {msg.content[:150]}")
    else:
        print(f"[{role}] {msg.content[:300]}")
print("-" * 60)
print(f"\n✅ Final Answer:\n{result['messages'][-1].content}")

C:\Users\likin\AppData\Local\Temp\ipykernel_15828\2272677627.py:27: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  advanced_agent = create_react_agent(


📋 Agent Message Trace:
------------------------------------------------------------
[HumanMessage] I need a brief report on what LangChain and LangGraph are. Search the knowledge base for each, then format the findings into a report titled 'AI Framework Overview'.
[AIMessage] 🔧 Calling: search_knowledge_base({'query': 'LangChain'})
[AIMessage] 🔧 Calling: search_knowledge_base({'query': 'LangGraph'})
[ToolMessage] ← LangChain is a framework for developing applications powered by LLMs. It supports chains, agents, and retrieval.
[ToolMessage] ← LangGraph is a library for building stateful, multi-actor applications with LLMs, built on top of LangChain.
[AIMessage] 🔧 Calling: format_report({'title': 'AI Framework Overview', 'sections': ['LangChain is a framework for developing applications powered by large language models (LLMs). It supports various structures such as chains, agents, and retrieval systems to facilitate the development process.', 'LangGraph is a library designed for building

---

## Summary

| Section | Feature | Key API |
|---------|---------|---------|
| 1 | Setup & Imports | `GenAIChatModel`, `GenAIEmbeddings` |
| 2 | Basic Chat | `model.invoke([HumanMessage(...)])` |
| 4 | Tool Calling | `model.bind_tools([...])`, `ToolMessage` |
| 5 | Structured Output | `model.with_structured_output(Schema)` |
| 6 | Embeddings | `embed_documents()`, `embed_query()` |
| 7 | Async | `ainvoke()`, `aembed_query()` |
| 8 | ReAct Agent | `create_react_agent(model, tools)` |
| 9 | Conversational Agent | `MemorySaver` + thread config |
| 10 | Advanced Agent | System prompt + multi-tool chaining |